## 1. Setup Environment

In [ ]:
# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install dependencies
!pip install torch torchvision timm opencv-python-headless pillow matplotlib seaborn scikit-learn tqdm ultralytics -q

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
import cv2
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import json
from ultralytics import YOLO
import random
import os

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Prepare Segmented Pod Dataset

In [ ]:
# Load YOLOv8 segmentation model from previous training
SEGMENTATION_MODEL_PATH = 'trained_models/cacao_segmentation_best.pt'

if not Path(SEGMENTATION_MODEL_PATH).exists():
    print("⚠️ Segmentation model not found. Please train YOLOv8 model first using train_cacao_segmentation_yolov8.ipynb")
    print("Or upload the trained model to 'trained_models/cacao_segmentation_best.pt'")
else:
    seg_model = YOLO(SEGMENTATION_MODEL_PATH)
    print(f"✅ Loaded segmentation model from {SEGMENTATION_MODEL_PATH}")

In [ ]:
# Function to extract segmented pods from images
def extract_segmented_pods(image_dir, output_dir, seg_model, conf_threshold=0.25):
    """
    Extract individual pod crops from images using segmentation model
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    image_paths = list(Path(image_dir).glob('*.jpg')) + list(Path(image_dir).glob('*.png'))
    print(f"Processing {len(image_paths)} images...")
    
    pod_count = 0
    metadata = []
    
    for img_path in tqdm(image_paths):
        # Run segmentation
        results = seg_model.predict(str(img_path), conf=conf_threshold, verbose=False)
        
        if len(results) == 0 or results[0].masks is None:
            continue
        
        # Load original image
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Extract each segmented pod
        masks = results[0].masks.data.cpu().numpy()
        boxes = results[0].boxes.xyxy.cpu().numpy()
        
        for idx, (mask, box) in enumerate(zip(masks, boxes)):
            # Get bounding box
            x1, y1, x2, y2 = map(int, box)
            
            # Resize mask to image size
            mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))
            
            # Apply mask to extract pod
            masked_img = img_rgb.copy()
            masked_img[mask_resized < 0.5] = 0  # Black background
            
            # Crop to bounding box
            pod_crop = masked_img[y1:y2, x1:x2]
            
            # Calculate morphological features
            area = np.sum(mask_resized > 0.5)
            perimeter = cv2.arcLength(cv2.findContours(
                (mask_resized > 0.5).astype(np.uint8),
                cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE
            )[0][0], True)
            
            width = x2 - x1
            height = y2 - y1
            aspect_ratio = width / height if height > 0 else 0
            
            # Save pod crop
            pod_filename = f"pod_{pod_count:05d}.jpg"
            pod_path = output_dir / pod_filename
            Image.fromarray(pod_crop).save(pod_path)
            
            # Store metadata
            metadata.append({
                'pod_id': pod_count,
                'filename': pod_filename,
                'source_image': img_path.name,
                'area': float(area),
                'perimeter': float(perimeter),
                'width': int(width),
                'height': int(height),
                'aspect_ratio': float(aspect_ratio),
            })
            
            pod_count += 1
    
    # Save metadata
    with open(output_dir / 'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n✅ Extracted {pod_count} pods to {output_dir}")
    return metadata

In [ ]:
# Extract pods from dataset
# Assuming you have the cacao dataset from Roboflow
DATASET_DIR = './cacao_dataset/train/images'  # Update this path
SEGMENTED_PODS_DIR = './segmented_pods'

if Path(SEGMENTATION_MODEL_PATH).exists():
    metadata = extract_segmented_pods(DATASET_DIR, SEGMENTED_PODS_DIR, seg_model)
    print(f"Total pods extracted: {len(metadata)}")
else:
    print("⚠️ Skipping pod extraction - segmentation model not found")

## 3. SimCLR Data Augmentation

In [ ]:
# SimCLR augmentation pipeline
class SimCLRAugmentation:
    def __init__(self, img_size=224):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(img_size, scale=(0.2, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    def __call__(self, x):
        return self.transform(x), self.transform(x)  # Two augmented views

# Standard transform for inference
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
# Dataset for SimCLR pre-training
class CacaoPodDataset(Dataset):
    def __init__(self, pod_dir, transform=None):
        self.pod_dir = Path(pod_dir)
        self.image_paths = list(self.pod_dir.glob('*.jpg'))
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            return self.transform(img)
        else:
            return test_transform(img)

# Create dataset
simclr_dataset = CacaoPodDataset(SEGMENTED_PODS_DIR, transform=SimCLRAugmentation())
print(f"Dataset size: {len(simclr_dataset)} pods")

## 4. MobileNetV3 with SimCLR

In [ ]:
# SimCLR Model with MobileNetV3 backbone
class SimCLRModel(nn.Module):
    def __init__(self, base_model='mobilenetv3_large_100', projection_dim=128):
        super(SimCLRModel, self).__init__()
        
        # Load MobileNetV3 from timm
        self.encoder = timm.create_model(base_model, pretrained=True, num_classes=0)
        
        # Get feature dimension
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224)
            feature_dim = self.encoder(dummy_input).shape[1]
        
        # Projection head for SimCLR
        self.projector = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, projection_dim)
        )
    
    def forward(self, x):
        features = self.encoder(x)
        projections = self.projector(features)
        return features, projections

# NT-Xent Loss (Normalized Temperature-scaled Cross Entropy)
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature
    
    def forward(self, z_i, z_j):
        batch_size = z_i.shape[0]
        
        # Normalize
        z_i = F.normalize(z_i, dim=1)
        z_j = F.normalize(z_j, dim=1)
        
        # Concatenate
        representations = torch.cat([z_i, z_j], dim=0)
        
        # Similarity matrix
        similarity_matrix = F.cosine_similarity(
            representations.unsqueeze(1),
            representations.unsqueeze(0),
            dim=2
        )
        
        # Create labels
        labels = torch.cat([
            torch.arange(batch_size) + batch_size,
            torch.arange(batch_size)
        ]).to(z_i.device)
        
        # Mask out self-similarity
        mask = torch.eye(2 * batch_size, dtype=torch.bool).to(z_i.device)
        similarity_matrix = similarity_matrix.masked_fill(mask, -9e15)
        
        # Compute loss
        similarity_matrix = similarity_matrix / self.temperature
        loss = F.cross_entropy(similarity_matrix, labels)
        
        return loss

model = SimCLRModel().to(device)
print(f"✅ Model initialized on {device}")
print(f"Encoder parameters: {sum(p.numel() for p in model.encoder.parameters()) / 1e6:.2f}M")

## 5. Train SimCLR

In [ ]:
# Training configuration
BATCH_SIZE = 64
EPOCHS = 100
LR = 3e-4
TEMPERATURE = 0.5

train_loader = DataLoader(
    simclr_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
criterion = NTXentLoss(temperature=TEMPERATURE)

print(f"Training setup:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LR}")
print(f"  Batches per epoch: {len(train_loader)}")

In [ ]:
# Training loop
history = {'loss': []}

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for (x_i, x_j) in pbar:
        x_i, x_j = x_i.to(device), x_j.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        _, z_i = model(x_i)
        _, z_j = model(x_j)
        
        # Compute loss
        loss = criterion(z_i, z_j)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})
    
    scheduler.step()
    
    avg_loss = epoch_loss / len(train_loader)
    history['loss'].append(avg_loss)
    
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, LR = {scheduler.get_last_lr()[0]:.6f}")
    
    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, f'simclr_checkpoint_epoch_{epoch+1}.pt')

print("\n✅ SimCLR training completed!")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history['loss'])
plt.title('SimCLR Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.savefig('simclr_training_loss.png')
plt.show()

## 6. Extract Features and Cluster Pods

In [ ]:
# Extract features from all pods
def extract_features(model, dataset, batch_size=32):
    model.eval()
    features_list = []
    
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting features"):
            if isinstance(batch, (list, tuple)):
                batch = batch[0]  # Handle SimCLR augmentation
            
            batch = batch.to(device)
            features, _ = model(batch)
            features_list.append(features.cpu().numpy())
    
    return np.vstack(features_list)

# Create dataset without augmentation
inference_dataset = CacaoPodDataset(SEGMENTED_PODS_DIR, transform=None)
features = extract_features(model, inference_dataset)

print(f"\n✅ Extracted features: {features.shape}")

In [ ]:
# Cluster pods into Low/Medium/High yield categories
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(features)

# Map clusters to yield categories (based on cluster centers)
# Assuming cluster with highest average feature values = high yield
cluster_means = [features[cluster_labels == i].mean() for i in range(n_clusters)]
cluster_ranking = np.argsort(cluster_means)  # Low to High

yield_categories = ['Low', 'Medium', 'High']
yield_mapping = {cluster_ranking[i]: yield_categories[i] for i in range(n_clusters)}

yield_labels = [yield_mapping[label] for label in cluster_labels]

print("\n📊 Yield Distribution:")
for category in yield_categories:
    count = yield_labels.count(category)
    print(f"  {category}: {count} pods ({count/len(yield_labels)*100:.1f}%)")

In [ ]:
# Visualize feature space with PCA
pca = PCA(n_components=2)
features_2d = pca.fit_transform(features)

plt.figure(figsize=(12, 8))
colors = {'Low': 'red', 'Medium': 'orange', 'High': 'green'}

for category in yield_categories:
    mask = np.array(yield_labels) == category
    plt.scatter(
        features_2d[mask, 0],
        features_2d[mask, 1],
        c=colors[category],
        label=category,
        alpha=0.6,
        s=50
    )

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Cacao Pod Feature Space (PCA)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('yield_clustering_pca.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nPCA explained variance: {pca.explained_variance_ratio_.sum():.2%}")

## 7. Siamese Ranking Network

In [ ]:
# Siamese Ranking Head
class YieldRankingModel(nn.Module):
    def __init__(self, encoder, feature_dim=1280, hidden_dim=256):
        super(YieldRankingModel, self).__init__()
        self.encoder = encoder
        
        # Freeze encoder (use pre-trained features)
        for param in self.encoder.parameters():
            param.requires_grad = False
        
        # Ranking head
        self.ranking_head = nn.Sequential(
            nn.Linear(feature_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Output: 0 = pod1 has higher yield, 1 = pod2 has higher yield
        )
    
    def forward(self, x1, x2):
        # Extract features
        with torch.no_grad():
            f1, _ = self.encoder(x1)
            f2, _ = self.encoder(x2)
        
        # Concatenate features
        combined = torch.cat([f1, f2], dim=1)
        
        # Predict ranking
        score = self.ranking_head(combined)
        
        return score

ranking_model = YieldRankingModel(model.encoder).to(device)
print(f"✅ Ranking model initialized")
print(f"Trainable parameters: {sum(p.numel() for p in ranking_model.parameters() if p.requires_grad) / 1e3:.2f}K")

In [ ]:
# Create synthetic ranking pairs using cluster labels
class RankingDataset(Dataset):
    def __init__(self, pod_dir, cluster_labels, transform=None):
        self.pod_dir = Path(pod_dir)
        self.image_paths = sorted(list(self.pod_dir.glob('*.jpg')))
        self.cluster_labels = cluster_labels
        self.transform = transform if transform else test_transform
        
        # Create ranking pairs
        self.pairs = []
        for _ in range(len(self.image_paths) * 3):  # 3x data augmentation
            idx1, idx2 = np.random.choice(len(self.image_paths), 2, replace=False)
            
            # Label: 1 if pod2 has higher yield, 0 otherwise
            label = 1.0 if cluster_labels[idx2] > cluster_labels[idx1] else 0.0
            
            self.pairs.append((idx1, idx2, label))
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        idx1, idx2, label = self.pairs[idx]
        
        img1 = Image.open(self.image_paths[idx1]).convert('RGB')
        img2 = Image.open(self.image_paths[idx2]).convert('RGB')
        
        return self.transform(img1), self.transform(img2), torch.tensor(label, dtype=torch.float32)

ranking_dataset = RankingDataset(SEGMENTED_PODS_DIR, cluster_labels)
ranking_loader = DataLoader(ranking_dataset, batch_size=32, shuffle=True, num_workers=4)

print(f"Ranking dataset: {len(ranking_dataset)} pairs")

In [ ]:
# Train ranking model
ranking_optimizer = torch.optim.Adam(ranking_model.ranking_head.parameters(), lr=1e-3)
ranking_criterion = nn.BCELoss()

RANKING_EPOCHS = 20
ranking_history = {'loss': [], 'accuracy': []}

for epoch in range(RANKING_EPOCHS):
    ranking_model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(ranking_loader, desc=f"Ranking Epoch {epoch+1}/{RANKING_EPOCHS}")
    for img1, img2, labels in pbar:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        
        ranking_optimizer.zero_grad()
        
        # Forward pass
        scores = ranking_model(img1, img2).squeeze()
        loss = ranking_criterion(scores, labels)
        
        # Backward pass
        loss.backward()
        ranking_optimizer.step()
        
        epoch_loss += loss.item()
        
        # Calculate accuracy
        predictions = (scores > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({'loss': loss.item(), 'acc': correct/total})
    
    avg_loss = epoch_loss / len(ranking_loader)
    accuracy = correct / total
    
    ranking_history['loss'].append(avg_loss)
    ranking_history['accuracy'].append(accuracy)
    
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}")

print("\n✅ Ranking model training completed!")

In [ ]:
# Plot ranking training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(ranking_history['loss'])
ax1.set_title('Ranking Model Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True)

ax2.plot(ranking_history['accuracy'])
ax2.set_title('Ranking Model Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.savefig('ranking_training.png')
plt.show()

## 8. Export Models

In [ ]:
# Save complete model
output_dir = Path('trained_models')
output_dir.mkdir(exist_ok=True)

# Save SimCLR encoder
torch.save({
    'encoder_state_dict': model.encoder.state_dict(),
    'projector_state_dict': model.projector.state_dict(),
    'cluster_labels': cluster_labels,
    'yield_mapping': yield_mapping,
    'kmeans': kmeans,
}, output_dir / 'simclr_encoder.pt')

# Save ranking model
torch.save({
    'ranking_head_state_dict': ranking_model.ranking_head.state_dict(),
    'accuracy': ranking_history['accuracy'][-1],
}, output_dir / 'ranking_model.pt')

# Save complete pipeline
torch.save({
    'encoder': model.encoder.state_dict(),
    'ranking_head': ranking_model.ranking_head.state_dict(),
    'yield_mapping': yield_mapping,
    'cluster_centers': kmeans.cluster_centers_,
}, output_dir / 'complete_yield_model.pt')

print("\n✅ Models saved to trained_models/")
print("  - simclr_encoder.pt")
print("  - ranking_model.pt")
print("  - complete_yield_model.pt")

In [ ]:
# Export to ONNX for mobile deployment
ranking_model.eval()

dummy_input1 = torch.randn(1, 3, 224, 224).to(device)
dummy_input2 = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(
    ranking_model,
    (dummy_input1, dummy_input2),
    output_dir / 'yield_ranking_model.onnx',
    input_names=['pod1', 'pod2'],
    output_names=['ranking_score'],
    dynamic_axes={
        'pod1': {0: 'batch'},
        'pod2': {0: 'batch'},
        'ranking_score': {0: 'batch'}
    }
)

print("\n✅ ONNX model exported: yield_ranking_model.onnx")

## 9. Model Summary

In [ ]:
# Create model info
model_info = {
    'model_name': 'Cacao Yield Estimation Pipeline',
    'architecture': 'MobileNetV3 + SimCLR + Siamese Ranking',
    'training_method': 'Self-supervised learning (no ground truth required)',
    'num_pods_trained': len(simclr_dataset),
    'simclr_epochs': EPOCHS,
    'ranking_epochs': RANKING_EPOCHS,
    'ranking_accuracy': float(ranking_history['accuracy'][-1]),
    'yield_categories': ['Low', 'Medium', 'High'],
    'input_size': [224, 224],
    'feature_dim': features.shape[1],
    'deployment_formats': ['PyTorch', 'ONNX'],
}

with open(output_dir / 'yield_model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("\n📊 Model Information:")
for key, value in model_info.items():
    print(f"  {key}: {value}")

print("\n✅ Model info saved to: trained_models/yield_model_info.json")

## 🎉 Training Complete!

### What We Built:
1. **YOLOv8 Segmentation**: Detects and segments cacao pods (trained in previous notebook)
2. **SimCLR Encoder**: Learns pod features without labels using self-supervised learning
3. **Siamese Ranking**: Compares two pods and predicts relative yield
4. **Clustering**: Groups pods into Low/Medium/High yield categories

### Deployment Pipeline:
```
Input Image → YOLOv8 Segmentation → Pod Crops → SimCLR Features → 
Ranking Model → Yield Prediction (Low/Medium/High)
```

### Next Steps:
1. Download models from `trained_models/` directory
2. Integrate into mobile app
3. Test on real field images
4. Fine-tune with farmer feedback

### Model Files:
- `cacao_segmentation_best.pt` - YOLOv8 segmentation model
- `complete_yield_model.pt` - Full yield estimation pipeline
- `yield_ranking_model.onnx` - ONNX format for mobile deployment

In [ ]:
# Automatically download trained models to your computer
import os
from pathlib import Path

# Check if running in Google Colab
try:
    from google.colab import files
    IN_COLAB = True
    print("✅ Running in Google Colab - Will auto-download models")
except ImportError:
    IN_COLAB = False
    print("💻 Not in Colab - Models saved locally")

# List of models to download
models_to_download = [
    'trained_models/simclr_encoder.pt',
    'trained_models/ranking_model.pt',
    'trained_models/complete_yield_model.pt',
    'trained_models/yield_ranking_model.onnx',
    'trained_models/model_info.json',
    'trained_models/yield_model_info.json',
]

if IN_COLAB:
    print("\n📥 Downloading models to your computer...")
    downloaded_count = 0
    for model_path in models_to_download:
        if os.path.exists(model_path):
            file_size = os.path.getsize(model_path) / 1e6
            print(f"\n⬇️ Downloading: {Path(model_path).name} ({file_size:.2f} MB)")
            try:
                files.download(model_path)
                print(f"   ✅ Downloaded successfully!")
                downloaded_count += 1
            except Exception as e:
                print(f"   ⚠️ Download failed: {e}")
        else:
            print(f"\n⚠️ Model not found: {model_path}")
    
    print("\n" + "="*60)
    print(f"🎉 {downloaded_count} models downloaded to your Downloads folder!")
    print("="*60)
    print("\n📌 Important files:")
    print("   • yield_ranking_model.onnx - For mobile app")
    print("   • complete_yield_model.pt - Full pipeline")
    print("   • yield_model_info.json - Model metadata")
else:
    print("\n📂 Models saved locally at:")
    for model_path in models_to_download:
        if os.path.exists(model_path):
            file_size = os.path.getsize(model_path) / 1e6
            print(f"  ✅ {model_path} ({file_size:.2f} MB)")
        else:
            print(f"  ❌ {model_path} (not found)")
    
    print("\n💡 To use in your mobile app:")
    print("   1. Copy yield_ranking_model.onnx to: mobile-app/assets/models/")
    print("   2. Copy complete_yield_model.pt to: public/models/")
    print("\n📖 Next steps:")
    print("   1. Download cacao_segmentation_best.onnx from previous notebook")
    print("   2. Add both ONNX models to mobile app")
    print("   3. Run: cd mobile-app && npx expo start")

## 10. Auto-Download Models (Google Colab)